In [1]:
import ee
import geemap
import json
import os
import requests
import ipyleaflet

In [2]:
#ee.Authenticate()
#ee.Initialize()

In [3]:
service_account = 'geemap-practice@gxe-gee.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'gxe-gee-ecce7a991ef0.json')
ee.Initialize(credentials)

In [4]:
ca_bound = ee.FeatureCollection('users/theaaronscherf/ca-state-boundary')
CA_Map = geemap.Map()
CA_Map.addLayer(ca_bound)

In [5]:
file_path = os.path.abspath('../own_cropland.json')

if not os.path.exists(file_path):
    url = 'https://github.com/AaronScherf/geemap_voila_ex/raw/master/own_cropland.geojson'
    r = requests.get(url)
    with open(file_path, 'w') as f:
        f.write(r.content.decode("utf-8"))        

with open(file_path) as f:
    json_data = json.load(f)

In [6]:
json_layer = GeoJSON(data=json_data, name='Farmland JSON', hover_style={'fillColor': 'red' , 'fillOpacity': 0.5})
CA_Map.add_layer(json_layer)
CA_Map

NameError: name 'GeoJSON' is not defined

In [ ]:
#CA_Map2 = geemap.Map()

In [ ]:
#ee_data = geojson_to_ee(json_data)
#CA_Map2.addLayer(ee_data, {}, "Farmland EE")

In [ ]:
roi = ca_bound

## NAIP: National Agriculture Imagery Program

The National Agriculture Imagery Program (NAIP) acquires aerial imagery during the agricultural growing seasons in the continental U.S.

NAIP projects are contracted each year based upon available funding and the FSA imagery acquisition cycle. Beginning in 2003, NAIP was acquired on a 5-year cycle. 2008 was a transition year, and a three-year cycle began in 2009.

NAIP imagery is acquired at a **one-meter** ground sample distance (GSD) with a horizontal accuracy that matches within six meters of photo-identifiable ground control points, which are used during image inspection.

Older images were collected using 3 bands (Red, Green, and Blue: RGB), but newer imagery is usually collected with an additional near-infrared band (RGBN). 

More information about NAIP imagery can be found on [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ).

## Create annual composite of NAIP imagery 

Select 4-band (RGBN) NAIP imagery.

In [ ]:
naip_ts = geemap.naip_timeseries(start_year=2009, end_year=2018, roi=roi)

Create a list of layer names to be shown under the dropdown list. 

In [ ]:
layer_names = ['NAIP ' + str(year) for year in range(2009, 2019)]
print(layer_names)

Set visualization parameters.

In [ ]:
naip_vis = {'bands': ['N', 'R', 'G']}

Create a split-panel map for visualizing NAIP imagery. Use the dropdown menus in the top corners to select the layer years.

In [ ]:
Map = geemap.Map()
Map.ts_inspector(left_ts=naip_ts, right_ts=naip_ts, left_names=layer_names, right_names=layer_names, left_vis=naip_vis, right_vis=naip_vis)
Map.centerObject(roi, zoom=6)
Map

## Create annual composite of Landsat imagery

Use the drawing tools to create an ROI

In [ ]:
#Input_Map = geemap.Map()
#Input_Map

In [ ]:
#region = Input_Map.draw_last_feature
#roi = region.geometry()

In [ ]:
landsat_ts = geemap.landsat_timeseries(roi=roi, start_year=1984, end_year=2019, start_date='01-01', end_date='12-31')

In [ ]:
layer_names = ['Landsat ' + str(year) for year in range(1984, 2020)]
print(layer_names)

In [ ]:
landsat_vis = {
    'min': 0,
    'max': 4000,
    'gamma': [1, 1, 1],
    'bands': ['NIR', 'Red', 'Green']}

In [ ]:
Landsat_Map = geemap.Map()
Landsat_Map.ts_inspector(left_ts=landsat_ts, right_ts=landsat_ts, left_names=layer_names, right_names=layer_names, left_vis=landsat_vis, right_vis=landsat_vis)
Landsat_Map.centerObject(roi, zoom=6)
Landsat_Map

## Compare Landsat imagery and National Land Cover Database (NLCD)

More information about NLCD can be found at the [Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD).

Use the drawing tools in the map below to select a region of interest.

In [ ]:
#Input_Map_2 = geemap.Map()
#Input_Map_2

In [ ]:
#region = Input_Map_2.draw_last_feature
#roi = region.geometry()

In [ ]:
NLCD = ee.ImageCollection('USGS/NLCD')

In [ ]:
NLCD_layers = NLCD.aggregate_array('system:id').getInfo()
print(NLCD_layers)

In [ ]:
NLCD2001 = ee.Image('USGS/NLCD/NLCD2001').select('landcover').clip(roi)
NLCD2006 = ee.Image('USGS/NLCD/NLCD2006').select('landcover').clip(roi)
NLCD2011 = ee.Image('USGS/NLCD/NLCD2011').select('landcover').clip(roi)
NLCD2016 = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(roi)

In [ ]:
NLCD = ee.ImageCollection([NLCD2001, NLCD2006, NLCD2011, NLCD2016])

In [ ]:
NLCD_layer_names = ['NLCD ' + str(year) for year in range(2001, 2017, 5)]
print(NLCD_layer_names)

In [ ]:
landsat_ts = geemap.landsat_timeseries(roi=roi, start_year=2001, end_year=2016, start_date='01-01', end_date='12-31')

In [ ]:
landsat_layer_names = ['Landsat ' + str(year) for year in range(2001, 2017)]

In [ ]:
landsat_vis = {
    'min': 0,
    'max': 4000,
    'gamma': [1, 1, 1],
    'bands': ['NIR', 'Red', 'Green']}

nlcd_vis = {
    'bands': ['landcover']
}

In [ ]:
Landsat_NLCD_Map = geemap.Map()
Landsat_NLCD_Map.ts_inspector(left_ts=landsat_ts, right_ts=NLCD, left_names=landsat_layer_names, right_names=NLCD_layer_names, left_vis=landsat_vis, right_vis=nlcd_vis)
Landsat_NLCD_Map.centerObject(roi, zoom=6)
Landsat_NLCD_Map